# Capstone: Sephora. Predicting prices based on Ingredients

## Problem description

It is an assumption customers make that their skin care product price is dependent on the ingredients in this product. The goal of my projects is to see if I can predict prices of the products based on the ingredients. To accomplish this goal, I first had to gather my data. I used Sephora.com data for this.

### Project Structure:
- Notebook 0. Selenium URL Collection
- Notebook 1. Saving data from URL to an HTML file
- Notebook 2. Collecting Product Data
- Notebook 3. Data Cleaning 
- Notebook 4. EDA
- Notebook 5. Fuzzy String Matching
- Notebook 6. Regression Modeling
- Notebook 7. Classification Modeling

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import groupby
from operator import itemgetter

/Users/yelenanevel/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
products = pd.read_csv('./data/products_clean_eda.csv')

In [3]:
products.head(2)

,name,brand,category,price,ingredients,no_reviews,hearts,size1,size2,url,final_size,price_per_ounce
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,moisturizing-cream-oils-mists,68.0,"Dicaprylyl Carbonate, Glycerin, Cetearyl Alcoh...",3000,216935,1.69,0.0,https://www.sephora.com/product/protini-tm-pol...,1.69,40.236686
1,The Water Cream,Tatcha,moisturizing-cream-oils-mists,68.0,"Dicaprylyl Carbonate, Glycerin, Cetearyl Alcoh...",2000,197492,0.00,1.7,https://www.sephora.com/product/the-water-crea...,1.70,40.000000


In [4]:
all_ingredients = []
#this code goes through all of the rows of ingredients and combines them in to one list
for i in products.index:
    list_ingredients = products.ingredients[i].split(', ') #splits a row ingredients into a list 
    for j in list_ingredients:     
        all_ingredients.append(j.strip().lower()) 

ingredients = pd.DataFrame([(x,all_ingredients.count(x)) for x in set(all_ingredients)],
                           columns = ['ingredient', 'count']).sort_values(by='ingredient')

ingredients = ingredients[ingredients.ingredient != '']
ingredients = ingredients[ingredients.ingredient != '1']

In [5]:
#https://stackoverflow.com/questions/17366788/python-split-list-based-on-first-character-of-word
letters = []
for letter, words in groupby(sorted(ingredients.ingredient), key=itemgetter(0)):
    for word in words:
        letters.append(letter)
letters

#first letter of the ingredient name
ingredients['letters'] = letters
ingredients.head()

,ingredient,count,letters
1376,10hydroxydecanoic acid,7,1
2949,110decanediol,1,1
1026,12hexanediol,313,1
1694,15 alkyl benzoate,2,1
788,1methylhydantoin2imide,2,1


In [6]:
#empty list that the rations will go in to
fuzzy = []
#looping through the letters of the alphabet
for i in sorted(set(ingredients['letters'])):
    #when ingredients starts with that letter
    df = ingredients[ingredients['letters'] == i]
    #calculating ratios 
    for k in df.ingredient:
        for j in df.ingredient:
            fuzzy.append((k, j, fuzz.ratio(k, j), df[df.ingredient == k]['count'].values[0],
                          df[df.ingredient == j]['count'].values[0]))
#creating a dataframe of ratios
fuzzy_ratio = pd.DataFrame(fuzzy, columns = ['ingredient1', 'ingredients2', 'ratio', 'word1 count', 'word2 count'])
fuzzy_ratio

,ingredient1,ingredients2,ratio,word1 count,word2 count
0,10hydroxydecanoic acid,10hydroxydecanoic acid,100,7,7
1,10hydroxydecanoic acid,110decanediol,46,7,1
2,10hydroxydecanoic acid,12hexanediol,35,7,313
3,10hydroxydecanoic acid,15 alkyl benzoate,15,7,2
4,10hydroxydecanoic acid,1methylhydantoin2imide,32,7,2
...,...,...,...,...,...
1712508,zizyphus jujuba seed extract,zingiber zerumbet extract,53,1,4
1712509,zizyphus jujuba seed extract,ziziphus jujuba fruit extract,81,1,10
1712510,zizyphus jujuba seed extract,zizyphus jujuba extract,90,1,2
1712511,zizyphus jujuba seed extract,zizyphus jujuba fruit extract,84,1,3


In [7]:
#only looking at top 80 ratio
fuzzy_ratio = fuzzy_ratio[fuzzy_ratio.ratio != 100]
top80=fuzzy_ratio[fuzzy_ratio.ratio > 80]
top80.head()

,ingredient1,ingredients2,ratio,word1 count,word2 count
63,30ethyl ascorbic acid,3oethyl ascorbic acid,95,1,24
93,3oethyl ascorbic acid,30ethyl ascorbic acid,95,24,1
105,4t butylcyclohexanol,4tbutylcyclohexanol,97,2,4
108,4tbutylcyclohexanol,4t butylcyclohexanol,97,4,2
1555,acacia dealbata flower,acacia dealbata flower cera,90,1,4


In [8]:
#checking for duplicates if the order of column is just reversed

#https://stackoverflow.com/questions/24676705/pandas-drop-duplicates-if-reverse-is-present-between-two-columns
top80['check_string'] = top80.apply(lambda row: ''.join(sorted([row['ingredient1'], row['ingredients2']])), axis=1)

#https://stackoverflow.com/questions/12497402/python-pandas-remove-duplicates-by-columns-a-keeping-the-row-with-the-highest
top80_nodupes = top80.groupby('check_string', group_keys=False).apply(lambda x: x.loc[x['word1 count'].idxmax()])
top80_nodupes

/Users/yelenanevel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ingredient1,ingredients2,ratio,word1 count,word2 count,check_string
check_string,,,,,,
30ethyl ascorbic acid3oethyl ascorbic acid,3oethyl ascorbic acid,30ethyl ascorbic acid,95,24,1,30ethyl ascorbic acid3oethyl ascorbic acid
4t butylcyclohexanol4tbutylcyclohexanol,4tbutylcyclohexanol,4t butylcyclohexanol,97,4,2,4t butylcyclohexanol4tbutylcyclohexanol
acacia dealbata floweracacia dealbata flower cera,acacia dealbata flower cera,acacia dealbata flower,90,4,1,acacia dealbata floweracacia dealbata flower cera
acacia senegalacacia senegal gum,acacia senegal gum,acacia senegal,88,35,7,acacia senegalacacia senegal gum
acacia victoriae extractacacia victoriae fruit extract,acacia victoriae extract,acacia victoriae fruit extract,89,2,1,acacia victoriae extractacacia victoriae fruit...
...,...,...,...,...,...,...
ziziphus jujuba fruit extractzizyphus jujuba fruit extract,ziziphus jujuba fruit extract,zizyphus jujuba fruit extract,97,10,3,ziziphus jujuba fruit extractzizyphus jujuba f...
ziziphus jujuba fruit extractzizyphus jujuba seed extract,ziziphus jujuba fruit extract,zizyphus jujuba seed extract,81,10,1,ziziphus jujuba fruit extractzizyphus jujuba s...
zizyphus jujuba extractzizyphus jujuba fruit extract,zizyphus jujuba fruit extract,zizyphus jujuba extract,88,3,2,zizyphus jujuba extractzizyphus jujuba fruit e...


In [9]:
#pivot table represents all the ingredients to keep vs the ones to change
pivot = top80_nodupes.pivot(index = 'ingredient1', columns = 'ingredients2', values = 'word1 count')
pivot

ingredients2,30ethyl ascorbic acid,4t butylcyclohexanol,acacia dealbata flower,acacia senegal,acacia victoriae fruit extract,acanthopanax senticosus extract,acer saccarum extract,acetyl glutamine,acetyl heptapeptide4,acetyl hexapeptide1,...,zinc sulphate,zingibeofficinale root extract,zingiber officianle root extract,zingiber officinale,zingiber officinale extract,zingiber officinale root e x tr act,zingiber officinale root oil,zizyphus jujuba extract,zizyphus jujuba fruit extract,zizyphus jujuba seed extract
ingredient1,,,,,,,,,,,,,,,,,,,,,
3oethyl ascorbic acid,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4tbutylcyclohexanol,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
acacia dealbata flower cera,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
acacia senegal gum,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
acacia victoriae extract,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zingiber officinale root extract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,58.0,58.0,NaN,58.0,58.0,58.0,NaN,NaN,NaN
zingiber officinale root oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
ziziphus jujuba fruit extract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,10.0


In [10]:
#based on the pivot table create a dataframe with columns to replace and value to replace with
ingredient_replace = []
for i in pivot.columns:
    #choosing the maximum number of counts in pivot column to replace with
    ingredient_replace.append((i, pivot[pivot[i] == pivot.loc[:, i].max()].index.values[0]))
ingredient_replace_df = pd.DataFrame(ingredient_replace, columns = ['to_replace', 'replace_with'])
ingredient_replace_df

,to_replace,replace_with
0,30ethyl ascorbic acid,3oethyl ascorbic acid
1,4t butylcyclohexanol,4tbutylcyclohexanol
2,acacia dealbata flower,acacia dealbata flower cera
3,acacia senegal,acacia senegal gum
4,acacia victoriae fruit extract,acacia victoriae extract
...,...,...
1795,zingiber officinale root e x tr act,zingiber officinale root extract
1796,zingiber officinale root oil,zingiber officinale root extract
1797,zizyphus jujuba extract,ziziphus jujuba fruit extract
1798,zizyphus jujuba fruit extract,ziziphus jujuba fruit extract


In [11]:
#need to replace now in the full data set
for i in products.index:
    split_list = products.loc[i, 'ingredients'].split(', ')
    for index, item in enumerate(split_list):
        if item in ingredient_replace_df['to_replace'].values:
            split_list[index] = ingredient_replace_df[ingredient_replace_df['to_replace']==item]['replace_with'].values[0]
    products.loc[i, 'ingredients'] = ', '.join(split_list)


In [12]:
products.to_csv('./data/products_fuzzywuzzy.csv', index = False)